In [1]:
import re
from sys import stdin
import os
import json

#folder containing the text files
documents_folder = '.\\texts'

#Regex necessary for identifying citations in text
author = "(?:[A-Z][A-Za-z'`-]+)"
etal = "(?:et al.?)"
additional = "(?:,? (?:(?:and |& )?" + author + "|" + etal + "))"
year_num = "(?:19|20)[0-9][0-9]"
page_num = "(?:, p.? [0-9]+)?"
year = "(?:, *" + year_num + page_num + "| *\(" + year_num + page_num + "\))"
name_year_regex = "(" + author + additional + "*" + year + ")"

#In case the regex type above is not used we assume the regex might look like this: [1] or (2)
num_bracket_regex = r"(?:\[\d+\])"

def get_docs(documents_folder):
    files = []
    for file_name in os.listdir(documents_folder):
        file_path = os.path.join(documents_folder, file_name)
        if os.path.isfile(file_path):
            files.append(file_path)
    return files


def check_if_sentence_matches(text, regex):
    pattern = re.compile(regex)
    match = pattern.search(text)
    return match is not None


def extract_text_from_file(text_file):
    """
    This funciton returns sentences and its corresponding text in case it is needed.

    Args:
        text_file (string): file path

    Returns:
        string: full text in file
        array: all sentences in text
    """
    print(f"Reading {text_file}")
    
    with open(text_file, 'r') as file:
        text = file.read().replace('\n', ' ')

    return text


def extract_citation_sections(text):
    sentences = re.split(r'[.!?](?=\s+[A-Z])', text)
    
    citation_sections = []
    prev_sen = None
    
    regex = name_year_regex + "|" + num_bracket_regex

    for sen, next_sen in zip(sentences, sentences[1:] + [None]):
        if check_if_sentence_matches(sen, regex):
            citation_section = {
                "citation_sentence": sen
            }
    
            if prev_sen and not check_if_sentence_matches(prev_sen, regex):
                citation_section = {
                    "previous_sentence": prev_sen,
                    "citation_sentence": sen
                }
        
            if next_sen and not check_if_sentence_matches(next_sen, regex):
                citation_section["next_sentence"] = next_sen
            
            citation_sections.append(citation_section)
        
        prev_sen = sen
    
    return citation_sections


def save_citation_sections(text_file, json_dir):
    text = extract_text_from_file(text_file)

    citation_sections = extract_citation_sections(text)

    # Save citation sections to a JSON file
    file_name = "citation_sections.json"
    json_data = {
        "id": "example",
        "sentences": citation_sections
    }

    file_path = os.path.join(json_dir, os.path.basename(text_file).replace(".txt",".json"))

    with open(file_path, "w") as file:
        json.dump(json_data, file, indent=4)

    print(f"Citation sections written to {file_path}")




docs = get_docs(documents_folder)

original_dir_name = ".\\jsonFiles"
cnt = 1
if os.path.exists(original_dir_name):
    while os.path.exists(f"{original_dir_name}_({cnt})"):
        cnt += 1
    original_dir_name = original_dir_name + f"_({cnt})"

os.makedirs(original_dir_name)

for doc in docs:
    print("_________________________________________________________________________________________________________________________")
    save_citation_sections(doc, original_dir_name)

_________________________________________________________________________________________________________________________
Reading .\texts\AburaedEtAl2017.txt
Citation sections written to .\jsonFiles\AburaedEtAl2017.json
_________________________________________________________________________________________________________________________
Reading .\texts\AburaedEtAl2018.txt
Citation sections written to .\jsonFiles\AburaedEtAl2018.json
_________________________________________________________________________________________________________________________
Reading .\texts\AburaedEtAl2020.txt
Citation sections written to .\jsonFiles\AburaedEtAl2020.json
_________________________________________________________________________________________________________________________
Reading .\texts\ChenEtAl2019.txt
Citation sections written to .\jsonFiles\ChenEtAl2019.json
_________________________________________________________________________________________________________________________
Read